# PLAN

- [x] Acquisition
    - [x] Select what list of repos to scrape.
    - [x] Get requests from the site.
    - [x] Save responses to csv.
- [x] Preparation
    - [x] Prepare the data for analysis.
- [ ] Exploration
    - [ ] Answer the following prompts:
        - [ ] What are the most common words in READMEs?
        - [ ] What does the distribution of IDFs look like for the most common words?
        - [ ] Does the length of the README vary by language?
        - [ ] Do different languages use a different number of unique words?
- [ ] Modeling
    - [ ] Transform the data for machine learning; use language to predict.
    - [ ] Fit several models using different text repressentations.
    - [ ] Build a function that will take in the text of a README file, and makes a prediction of language.
- [ ] Delivery
    - [ ] Github repo
        - [x] This notebook.
        - [ ] Documentation within the notebook.
        - [ ] README file in the repo.
        - [ ] Python scripts if applicable.
    - [ ] Google Slides
        - [ ] 1-2 slides only summarizing analysis.
        - [ ] Visualizations are labeled.
        - [ ] Geared for the general audience.
        - [ ] Share link @ readme file and/or classroom.

# ENVIRONMENT

In [1]:
import os
import sys

import pandas as pd
import re
import json
import unicodedata
import nltk
import spacy

from requests import get
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import prepare

ADDITIONAL_STOPWORDS = ['readme', '\n\n\n']

# ACQUIRE

In [2]:
# We have decided to search Github for "san antonio data" and scrape the results.
# https://github.com/open-austin

In [3]:
def get_github_repo(url):
    """
    This function takes a url and returns a dictionary that
    contains the content and language of the readme file.
    """
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    readme = soup.find('div', id='readme')
    language = soup.find('span', class_='lang')
    
    d = dict()
    if readme is None:
        d['readme'] = 'No readme file.'
    else:
        d['readme'] = readme.text
    if language is None:
        d['language'] = 'No language specified.'
    else:
        d['language'] = language.text
    return d

In [4]:
# This line to test out the function.
get_github_repo('https://github.com/open-austin/atx-citysdk-js')

{'readme': '\n\n\n\n        README.md\n      \n\n\nCitySDK Austin Parks\nThis is a demonstration app forked from Austin Park Equity to give a mapping example using the CitySDK from Census.gov. We\'d love for you to try to plug in data from your city.\nSee what the live Austin demo lookes like.\nWhat is the CitySDK?\nCitySDK is a toolbox for civic innovators to connect local and national public data developed by the US Census Department. You should explore their wonderful guides and documentation.\n\nCitySDK\nCitySDK Guides\nCitySDK Code Examples\n\nRequests\nHere is an example of the request we are making for demographic data in Austin, Texas (Travis County):\nvar sdk = new CitySDK();\nvar censusModule = sdk.modules.census;\ncensusModule.enable(config.citySDK_token);\n\nvar request = {\n  "lat": config.city_lat,\n  "lng": config.city_lng,\n  "level": "county",\n  "sublevel": "true",\n  "api" : "acs5",\n  "variables": [\n    "population",  // Total Population\n    "income",  // Median I

In [5]:
def get_github_links(url):
    """
    This function takes in a url and returns a list of links
    that comes from each individual repo listing page.
    """
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.findAll('a', itemprop='name codeRepository', attrs={'href': re.compile("^/")}):
        links.append(link.get('href'))
    return links

In [6]:
# This line to test out the function.
get_github_links('https://github.com/open-austin?page=3')

['/open-austin/onebusaway-docker',
 '/open-austin/leaflet-talk',
 '/open-austin/open-austin-logo',
 '/open-austin/open-data-docs',
 '/open-austin/austin-park-equity',
 '/open-austin/ship-it-2015',
 '/open-austin/austin-parks-photos',
 '/open-austin/demo-website',
 '/open-austin/tecfiler',
 '/open-austin/liberate-the-data',
 '/open-austin/atx-citysdk-js',
 '/open-austin/mybuildingdoesntrecycle',
 '/open-austin/voteatx-svc',
 '/open-austin/hackathon-ideas',
 '/open-austin/austin-recycles',
 '/open-austin/aac-pets-feed',
 '/open-austin/austingreenmap-cordova',
 '/open-austin/council-connect',
 '/open-austin/stolen-bikes',
 '/open-austin/vanilla-rails',
 '/open-austin/OARK-back-end',
 '/open-austin/hack-projects',
 '/open-austin/hack-team-projects-list']

In [7]:
def get_all_github_links(path, num_pages):
    """
    This function takes in a url path and number of pages
    and returns a list of lists of all links.
    """
    all_links = []
    for i in range(num_pages):      # Number of pages plus one
        page = i + 1
        response = get(path + str(page))
        soup = BeautifulSoup(response.text, 'html.parser')
        all_links.append(get_github_links(path + '?page=' + str(page)))
    return all_links

In [8]:
# This line to test out the function.
get_all_github_links('https://github.com/open-austin', 3)

[['/open-austin/influence-texas',
  '/open-austin/open-austin.github.io',
  '/open-austin/harris-county-bookings',
  '/open-austin/ballotapi',
  '/open-austin/iced-coffee',
  '/open-austin/data-portal-analysis',
  '/open-austin/budgetparty',
  '/open-austin/complaint-map',
  '/open-austin/lobbying-in-austin',
  '/open-austin/pet-finder',
  '/open-austin/open-carry',
  '/open-austin/project-ideas',
  '/open-austin/hack-task-aggregator',
  '/open-austin/data-open-austin-org',
  '/open-austin/Civic_Project',
  '/open-austin/construction-permits',
  '/open-austin/awesome-austin',
  '/open-austin/GreenBelts',
  '/open-austin/atx-restaurant-scores',
  '/open-austin/fake-the-news',
  '/open-austin/consumer-protection',
  '/open-austin/transitime-docker',
  '/open-austin/Restaurant-Health-Inspection-Score-Prediction',
  '/open-austin/government.github.com',
  '/open-austin/sporkability',
  '/open-austin/open-data-progress-report',
  '/open-austin/water-quality',
  '/open-austin/budgetparty-lan

In [9]:
def traverse(o, tree_types=(list, tuple)):
    if isinstance(o, tree_types):
        for value in o:
            for subvalue in traverse(value, tree_types):
                yield subvalue
    else:
        yield o

In [10]:
def get_github_readme(url, num_pages, cache=True):
    if cache and os.path.exists('github_readme.json'):
        readme_text = json.load(open('github_readme.json'))
    else:
        data = get_all_github_links(url, num_pages)
        readme_text = []
        for value in traverse(data):
            print('https://github.com'+value)
            readme_text.append(get_github_repo('https://github.com' + value))
        json.dump(readme_text, open('github_readme.json', 'w'))
    return readme_text

In [11]:
# Bringing it all together chaining...
corpus = get_github_readme('https://github.com/open-austin', 3, cache=True)
corpus

[{'readme': '\n\n\n\n        README.md\n      \n\n\nInfluence Texas has launched!  Checkout the live webapp at https://app.influencetexas.com/\nFind more information at https://www.influencetexas.com/\nWelcome & Project Summary\nWelcome! We‚Äôre so glad you‚Äôve found your way to INFLUENCE TX. This project was started at ATX Hack for Change June 2-4, 2017 by Amy M. Mosley, a former investigative reporter who was frustrated by a fruitless search for fact-based, unbiased sources for information during the toxic 2016 election cycle. In 2017 all ATX Hack for Change projects were aligned with United Nations Sustainable Development Goals. This project falls under Goal 16: Promote just, peaceful and inclusive societies.\nThe premise is this, Politicians lie.\nAnd usually, they do whatever they get paid to do. You need to know:\nWho is paying them?\nHow are they voting as a result?\nBy linking lawmakers‚Äô campaign finance records to their voting records, taxpayers can track the influence of m

# PREPARE

In [12]:
def basic_clean(original):
    word = original.lower()
    word = unicodedata.normalize('NFKD', word)\
                                .encode('ascii', 'ignore')\
                                .decode('utf-8', 'ignore')
    word = re.sub(r"[^a-z0-9'\s]", '', word)
    word = word.replace('\n',' ')
    word = word.replace('\t',' ')
    return word

def tokenize(original):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(basic_clean(original))

def stem(original):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in original.split()]
    original_stemmed = ' '.join(stems)
    return original_stemmed

def lemmatize(original):
    nlp = spacy.load('en', parse=True, tag=True, entity=True)
    doc = nlp(original) # process the text with spacy
    lemmas = [word.lemma_ for word in doc]
    original_lemmatized = ' '.join(lemmas)
    return original_lemmatized

def remove_stopwords(original, extra_words=['readmemd'], exclude_words=[]):
    tokenizer = ToktokTokenizer()

    stopword_list = stopwords.words('english')

    for word in extra_words:
        stopword_list.append(word)
    for word in exclude_words:
        stopword_list.remove(word)

    words = original.split()
    filtered_words = [w for w in words if w not in stopword_list]

    print('Removed {} stopwords'.format(len(words) - len(filtered_words)))
    print('---')

    original_nostop = ' '.join(filtered_words)

    return original_nostop

def prep_article(article):
    
    article_stemmed = stem(basic_clean(article['readme']))
    article_lemmatized = lemmatize(article_stemmed)
    article_without_stopwords = remove_stopwords(article_lemmatized)
    
    article['stemmed'] = article_stemmed
    article['lemmatized'] = article_lemmatized
    article['clean'] = article_without_stopwords
    
    return article

def prepare_article_data(corpus):
    transformed  = []
    for article in corpus:
        transformed.append(prep_article(article))
    return transformed

In [22]:
df = pd.DataFrame(prepare_article_data(corpus))
df.shape

Removed 274 stopwords
---
Removed 51 stopwords
---
Removed 85 stopwords
---
Removed 64 stopwords
---
Removed 47 stopwords
---
Removed 217 stopwords
---
Removed 168 stopwords
---
Removed 46 stopwords
---
Removed 44 stopwords
---
Removed 162 stopwords
---
Removed 34 stopwords
---
Removed 184 stopwords
---
Removed 142 stopwords
---
Removed 24 stopwords
---
Removed 72 stopwords
---
Removed 51 stopwords
---
Removed 30 stopwords
---
Removed 4 stopwords
---
Removed 3 stopwords
---
Removed 127 stopwords
---
Removed 2 stopwords
---
Removed 23 stopwords
---
Removed 44 stopwords
---
Removed 134 stopwords
---
Removed 261 stopwords
---
Removed 96 stopwords
---
Removed 101 stopwords
---
Removed 6 stopwords
---
Removed 78 stopwords
---
Removed 15 stopwords
---
Removed 7 stopwords
---
Removed 144 stopwords
---
Removed 387 stopwords
---
Removed 70 stopwords
---
Removed 43 stopwords
---
Removed 66 stopwords
---
Removed 75 stopwords
---
Removed 35 stopwords
---
Removed 207 stopwords
---
Removed 3 stopwor

(83, 5)

In [14]:
df = df[['clean', 'language']]
df

clean                language
0   influenc texa ha launch checkout live webapp h...                  Python
1   openaustinorg open austin websit draft blog po...                    HTML
2   harriscountybook searchabl repositori aggreg d...                  Python
3   ballotapi ballot develop note thi project deve...  No language specified.
4   ice coffe thi -PRON- keep track -PRON- brigad ...  No language specified.
5   datum portal analysi ' lot content citi austin...                  Python
6   tabl content project descript ' next dev get s...              JavaScript
7   complaintmap map tag element legal complaint t...        Jupyter Notebook
8   lobbyinginaustin visual austin lobbyist regist...        Jupyter Notebook
9   pet alert part atx hack chang 2014 problem cur...                    Ruby
10  open carri api goal thi project creat api serv...                    Ruby
11  open austin project idea want start open austi...  No language specified.
12  hack task aggreg web applic aggreg task across...              JavaScript
13  readmetxt thi site contain datum publish open ...                    Ruby
14  civic project thi project contain datum civic ...                    Ruby
15  citi austin construct permit searchabl databa ...                  Python
16  awesomeaustin collect awesom austin civic tech...  No language specified.
17  greenbelt interact web map app chat share geot...                     CSS
18  atxrestaurantscor app queri austin restaur sco...                    HTML
19  partner organ mozilla hive austin austin monit...  No language specified.
20  consumerprotect visual consum financi protect ...                    HTML
21  transitimedb httptransitimehostcloudappnetapiv...                   Shell
22  restauranthealthinspectionscorepredict problem...  No language specified.
23  github govern gather curat featur stori peopl ...                    HTML
24  sporkabl best practic make project easi fork i...  No language specified.
25  open datum progress report overview open austi...                    HTML
26  austin creek water qualiti problem -PRON- hard...  No language specified.
27  budgetpartyland land page hub thing budget par...              JavaScript
28  restroommap thi base boilerpl mobil hybrid app...              JavaScript
29  hackpad thi repo contain open austin hackpad d...  No language specified.
..                                                ...                     ...
53  open default open austin draft prioriti open d...  No language specified.
54  pull datum portal catalog pull metatdata datas...              JavaScript
55  captur repo answer question public transit aus...        Jupyter Notebook
56  civictechcanva version busi model canva -PRON-...                    HTML
57  hacksummitprojectplan thi document need write ...  No language specified.
58                                    codeacross 2016  No language specified.
59  meetupproxyawslambda proxi meetupcom api live ...              JavaScript
60  onebusawaydock look httpsgithubcomopenaustinon...  No language specified.
61  leaflet lightn talk power httpsgithubcomhakime...              JavaScript
62  openaustinlogo thi project contain open austin...  No language specified.
63  austin ' open datum initi thi repo use develop...                    HTML
64  austin park equiti app thi civic app aim help ...              JavaScript
65  ship -PRON- weekend 2015 list project project ...  No language specified.
66  austin park photo scraper thi project provid p...              JavaScript
67  open austin demo websit thi packag contain dem...  No language specified.
68  tecfil util gener campaign financ file candida...                    Ruby
69  liber datum collect dataset commun would like ...  No language specified.
70  citysdk austin park thi demonstr app fork aust...              JavaScript
71  -PRON- build recycl crowdsourc datum residenti...              JavaScript
72  readmerdoc voteatx servic voteatx vote place f...                 

In [15]:
languages = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
languages.columns = ['n', 'ratio']
languages

n     ratio
No language specified.  26  0.313253
JavaScript              22  0.265060
HTML                    10  0.120482
Ruby                     9  0.108434
Python                   6  0.072289
Jupyter Notebook         4  0.048193
Shell                    3  0.036145
CSS                      2  0.024096
PureBasic                1  0.012048

In [24]:
# removing all rows that has 'No language specified.'
df = df[df.language != 'No language specified.']

In [25]:
df.shape

(57, 5)

In [26]:
languages = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
languages.columns = ['n', 'ratio']
languages

n     ratio
JavaScript        22  0.385965
HTML              10  0.175439
Ruby               9  0.157895
Python             6  0.105263
Jupyter Notebook   4  0.070175
Shell              3  0.052632
CSS                2  0.035088
PureBasic          1  0.017544

# EXPLORE

# MODEL